In [1]:
%pylab inline 
import pandas as pd 
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_data = []
with open('../system/user_study_2017_12_08.log') as f:
    for line in f:
        lsp = line.split(',')
        date = lsp[0]
        dataset_name = lsp[1][1:-1]
        clicks = json.loads(",".join(lsp[2:]))
        if dataset_name!="default":
            dataset = json.loads(open("../system/static/generated_dashboards/"+dataset_name).read().replace("\\",""))
            for nodeID in dataset.keys():
                df_data.append([dataset_name,nodeID,clicks[nodeID],dataset[nodeID][-1]["filter"],dataset[nodeID][-1]["populationSize"]])

In [3]:
df = pd.DataFrame(df_data,columns=["dataset_name","nodeID","clicks","filters","populationSize"])

In [4]:
df.to_csv("user_click_data.csv",index=None)